In [1]:
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
import gspread
import io
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
from apiclient import discovery
from apiclient.http import MediaFileUpload
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import googleapiclient.discovery
from itertools import islice

import sa_drive as drive

"sa_drive" module does all the interaction with Google Drive API Using Service Account to authenticate.

Note: A service Account acts as a "normal" Gsuite account, which means it has it's own "drive space", so you need to share the files or directories you want "service account" to see.

You can access Gsuite information in two ways:

service accounts (using SAs)
user accounts (using Oauth)
We use in this example Service Accounts because we want to use Cloud Function to be triggered without user input, which happens with Oauth (requires user browser authentication)

In [ ]:
files = drive.get_sibs_files()
print(len(files))
# print(files)

In [ ]:
files = pd.DataFrame(files)
files = files.sort_values(by ='name')
cols = list(files.columns)
a, b = cols.index('id'), cols.index('name')
cols[b], cols[a] = cols[a], cols[b]
files = files[cols]

files.head()

In [ ]:

# print(len(files.index))

# for name, x in files.iterrows():
    # print('name: {}, id: {}'.format(x['name'], x['id']))

In [ ]:
# for row in files.itertuples():
#    print('name: {}, id: {}'.format(row.name, row.id))


In [ ]:
LIMIT = 2
for row in islice(files.itertuples(), LIMIT):
    print('name: {}, id: {}'.format(row.name, row.id))


In [6]:
widths = [1,2,1,4,8,12,1,10,5,15,10,5, 9, 9, 8, 1, 1, 12, 6]
header = ['reg', 'cod', 'PDD', 'IdSIBS', 'numSIBS','date', 'typeT', 'idT', 'idTrans', 'LocT', 'value', 'tarif', 'refMB', 'NIF', 'nFact', 'modCom', 'CodEmp', 'RespID', 'filler']
# df = pd.read_fwf('202002040340.inp', widths=widths, skipfooter=1) 
# df.columns = header


In [12]:
# df = pd.read_fwf(io.StringIO(strIO.decode("utf-8")), widths=widths, skipfooter=1, header=None)
f = open('201902050161.inp', "rb")
print(type(f))

df = pd.read_fwf('201902050161.inp', widths=widths, skipfooter=1, header=None)
df.columns = header
df.head()

<class '_io.BufferedReader'>


,reg,cod,PDD,IdSIBS,numSIBS,date,typeT,idT,idTrans,LocT,value,tarif,refMB,NIF,nFact,modCom,CodEmp,RespID,filler
0,0,MS,T,M000,10334,221201902051,2,160128110,497,823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,04,F,0660,125970,201902051636,A,18807101,8255,CANTANHEDE,1000.0,0.0,111111111.0,0.0,1.0,0.0,0.0,NaN,NaN


In [21]:
ioBytes = io.BytesIO()
ioBytes.write(b'0MSTM000000103342212019092712019092511000497823                                                                         \r\n104G036400077772201909271553A003370038033055MAIA           0000000900000001111111110000000000000012300                  \r\n104F036300170810201909271702A003506970594033LISBOA         0000001000000001111111110000000000000012400                  \r\n90000000200000000000001900000000000000000000000000                                                                      \r\n')
print(type(ioBytes))
df = pd.read_fwf(ioBytes, widths=widths, skipfooter=1, header=None)

# ioBytes.seek(0)
# print(ioBytes.getvalue())


<class '_io.BytesIO'>


EmptyDataError: No columns to parse from file

In [28]:
for row in islice(files.itertuples(), LIMIT):
    file = drive.read_file(row.id)
   
    print(row.name)
    print(type(file))
    # print(file.getvalue())
    df = pd.read_fwf(file.getvalue(), widths=widths, skipfooter=1, header=None)
    df.columns = header

Download 100%.
201902050161.inp
<class '_io.StringIO'>


OSError: [Errno 63] File name too long: '0MSTM000000103342212019020512016012811000497823                                                                         \r\n104F066000125970201902051636A001880710108255CANTANHEDE     0000001000000001111111110000000000000000100                  \r\n90000000100000000000001000000000000000000000000000                                                                      \r\n'

In [254]:
df['date']= pd.to_duu

In [258]:
df.head()

,reg,cod,PDD,IdSIBS,numSIBS,date,typeT,idT,idTrans,LocT,value,tarif,refMB,NIF,nFact,modCom,CodEmp,RespID,filler
0,1,4,2,460,363385,2020-02-04 08:37:00,P,0,0,MBWAY,200,0,111111111,240571215,41,0,0,NaN,NaN
1,1,4,2,461,663982,2020-02-04 15:53:00,P,0,0,MBWAY,1000,0,111111111,0,42,0,0,NaN,NaN


In [259]:
excelFrame = df[['date', 'NIF', 'value', 'LocT']]

In [262]:
excelFrame.head()

,date,NIF,value,LocT
0,2020-02-04 08:37:00,240571215,200,MBWAY
1,2020-02-04 15:53:00,0,1000,MBWAY


In [264]:
writer = ExcelWriter('SIBS_exporter.xlsx')
excelFrame.to_excel(writer,'test')
writer.save()

In [12]:
widths = [1, 4, 7, 7, 9, 9, 5, 3, 2, 73]
header = ['reg', 'type', 'IDori', 'IDdest', 'lastDate', 'IDlast', 'entity', 'coin', 'VAT', 'filler']
file_info = pd.read_fwf('202002040340.inp', widths=widths, nrows=1, header=None) 
file_info.columns = header
file_info.head()


,reg,type,IDori,IDdest,lastDate,IDlast,entity,coin,VAT,filler
0,0,MSTM,1,334221,202002041,202002031,10004,978,23,NaN


In [11]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('SA_sheet-write-access.json', scope)
client = gspread.authorize(creds)

sheet = client.open('Helpo-SIBS').sheet1

data = sheet.get_all_records()
sheet_df = pd.DataFrame(data)
sheet_df.head()

SpreadsheetNotFound: 